In [ ]:
import tifffile
import numpy as np
from tqdm import tqdm
from pathlib import Path
import os
import pandas as pd
import sys
sys.path.append(r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\10_code\UTvsXCT-preprocessing')
import dbtools as db
from preprocess_tools import io, aligner, reslicer

# Database conection

In [2]:
try:
    conn = db.connect()
    print("Connected to the database")
except Exception as error:
    print(error)

Connected to the database


# Data retrieval

We have to load the data from the database to get:

1. The file ids to use them as parent measurement

## Measurements file paths

In [3]:
original_paths = [Path(r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Juan Ignacio\probetas\4\volume_eq_rotated'),
                  Path(r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Juan Ignacio\probetas\5\volume_eq_rotated'),
                  Path(r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Juan Ignacio\probetas\7\volume_eq'),
                  Path(r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Juan Ignacio\probetas\8\volume_eq'),
                  Path(r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Juan Ignacio\probetas\10\volume_eq'),
                  Path(r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Juan Ignacio\probetas\11\volume_eq'),
                  Path(r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Juan Ignacio\probetas\12\volume_eq')]

measurements_table = db.relation_metadata('measurements','samples','sample_measurements')

parent_id_column = 'measurementtype_id_measurement'

# for each original path, get the id_measurement where file_path_measurement is equal to the str of the original path

original_ids = []
original_measurementtype_ids = []
original_sample_names = []

for original_path in original_paths:
    original_path_str = str(original_path)
    original_id = measurements_table.loc[measurements_table['file_path_measurement'] == original_path_str, 'id_measurement'].values[0]
    original_measurementtype_id = measurements_table.loc[measurements_table['file_path_measurement'] == original_path_str, parent_id_column].values[0]
    original_sample_name = measurements_table.loc[measurements_table['file_path_measurement'] == original_path_str, 'name_sample'].values[0]
    original_ids.append(original_id)
    original_measurementtype_ids.append(original_measurementtype_id)
    original_sample_names.append(original_sample_name)
    print(f"Original path: {original_path_str}, ID: {original_id}", 
          f"Measurement type ID: {original_measurementtype_id}",
          f"Sample name: {original_sample_name}")

Original path: \\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Juan Ignacio\probetas\4\volume_eq_rotated, ID: 60 Measurement type ID: 2 Sample name: JI_4
Original path: \\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Juan Ignacio\probetas\5\volume_eq_rotated, ID: 61 Measurement type ID: 2 Sample name: JI_5
Original path: \\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Juan Ignacio\probetas\7\volume_eq, ID: 28 Measurement type ID: 2 Sample name: JI_7
Original path: \\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Juan Ignacio\probetas\8\volume_eq, ID: 29 Measurement type ID: 2 Sample name: JI_8
Original path: \\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Juan Ignacio\probetas\10\volume_eq, ID: 31 Measurement type ID: 2 Sample name: JI_10
Original path: \\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Juan Ignacio\probetas\11\volume_eq, ID: 32 Measurement type ID: 2 Sample name: JI_11
Original

## Functions

## Database loading

In [4]:
def load_measurement(file,file_path,parent_id,measurementtype_id,sample_name):

    parent_id_column = 'measurementtype_id'

    sample_names = [sample_name]

    main_parameters = {'file_path':file_path,parent_id_column:measurementtype_id,'parent_measurement_id':parent_id}

    # metadata

    #for each parameter of the measurement a metadata has to be created

    metadata_parameters = []

    #dimensions
    metadata_parameters.append({'key':'height', 'value':str(file.shape[0]), 'type':'cardinal'})

    metadata_parameters.append({'key':'width', 'value':str(file.shape[1]), 'type':'cardinal'})

    metadata_parameters.append({'key':'depth', 'value':str(file.shape[2]), 'type':'cardinal'})

    #dtype

    metadata_parameters.append({'key':'dtype', 'value':str(file.dtype), 'type':'nominal'})

    #file type

    metadata_parameters.append({'key':'file_type', 'value':'folder', 'type':'nominal'})

    #aligned

    metadata_parameters.append({'key':'aligned', 'value':'True', 'type':'boolean'})

    #equalized

    metadata_parameters.append({'key':'equalized', 'value':'True', 'type':'boolean'})

    #axes
    metadata_parameters.append({'key':'axes', 'value':'z,x,y', 'type':'nominal'})

    #transformation interpolation order
    metadata_parameters.append({'key':'transformation_interpolation_order', 'value':'cubic', 'type':'nominal'})

    print('Parameters to be inserted: ')
    for key, value in main_parameters.items():
        print(f"-    {key}: {value}")

    table_name = 'measurements'

    # Extract column names and values from the attributes dictionary
    columns = ', '.join(main_parameters.keys())
    values = ', '.join([f"'{v}'" for v in main_parameters.values()])

    # Construct the SQL INSERT statement
    sql = f"INSERT INTO {table_name} ({columns}) VALUES ({values})"

    print(sql)

    # Create a cursor object using the cursor() method
    cursor = conn.cursor()

    # Execute the SQL statement
    cursor.execute(sql)

    cursor.execute('COMMIT')

    cursor.close()

    data = db.get_data(table_name)

    data[data['file_path_measurement'] == str(file_path)]

    row_id = data['id_measurement'].values[-1]

    metadata_table_name =  table_name[:-1] + '_metadata'

    for attributes in metadata_parameters:

        attributes[table_name[:-1] + '_id'] = row_id

        # Extract column names and values from the attributes dictionary
        columns = ', '.join(attributes.keys())
        values = ', '.join([f"'{v}'" for v in attributes.values()])

        # Construct the SQL INSERT statement
        sql = f"INSERT INTO {metadata_table_name} ({columns}) VALUES ({values})"

        print(sql)

        # Create a cursor object using the cursor() method
        cursor = conn.cursor()

        # Execute the SQL statement
        cursor.execute(sql)

        cursor.execute('COMMIT')

        cursor.close()
    
    samples_data = db.get_data_metadata('samples')

    #get the ids of the samples in sample_names
    samples_data = samples_data[samples_data['name_sample'].isin(sample_names)]

    sample_ids = samples_data['id_sample'].values.tolist()

    relational_table_name = 'sample_measurements'

    for sample_id in sample_ids:

        relational_parameters = {'sample_id': sample_id, 'measurement_id': row_id}

        # Extract column names and values from the attributes dictionary
        columns = ', '.join(relational_parameters.keys())
        values = ', '.join([f"'{v}'" for v in relational_parameters.values()])

        # Construct the SQL INSERT statement
        sql = f"INSERT INTO {relational_table_name} ({columns}) VALUES ({values})"

        print(sql)

        # Create a cursor object using the cursor() method
        cursor = conn.cursor()

        # Execute the SQL statement
        cursor.execute(sql)

        cursor.execute('COMMIT')

        cursor.close()

## main loop

For each file:

1. Load it

2. Align it

3. Reslice it

4. Save it

5. Save it to the database

In [ ]:
for i in range(len(original_paths)):
    # Get the original path and ID
    original_path = original_paths[i]
    original_id = original_ids[i]
    original_measurementtype_id = original_measurementtype_ids[i]
    original_sample_name = original_sample_names[i]

    #load the volume
    volume = io.load_tif(original_path)

    #align the volume
    volume = aligner.main(volume, order=3)

    #reslice the volume
    volume = reslicer.rotate_90(volume)
    volume = reslicer.reslice(volume,'Top')

    #save the volume
    #save it in the parent folder of it with the name aligned_90rotleft_reslicebottom.tif
    save_path = original_path.parent / f"aligned_90rotright_reslicetop.tif"
    #save in the path
    io.save_tif(save_path, volume)

    #update the database
    load_measurement(volume,save_path,original_id,original_measurementtype_id,original_sample_name)

Progress: 100%|██████████| 3224/3224 [00:37<00:00, 86.84it/s]


computing otsu
Transforming
computing otsu
Bounding box coordinates: zmin=2, zmax=3284, ymin=98, ymax=1632, xmin=60, xmax=262
Parameters to be inserted: 
-    file_path: \\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Juan Ignacio\probetas\4\aligned_90rotright_reslicetop.tif
-    measurementtype_id: 2
-    parent_measurement_id: 60
INSERT INTO measurements (file_path, measurementtype_id, parent_measurement_id) VALUES ('\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Juan Ignacio\probetas\4\aligned_90rotright_reslicetop.tif', '2', '60')
INSERT INTO measurement_metadata (key, value, type, measurement_id) VALUES ('height', '203', 'cardinal', '84')
INSERT INTO measurement_metadata (key, value, type, measurement_id) VALUES ('width', '3283', 'cardinal', '84')
INSERT INTO measurement_metadata (key, value, type, measurement_id) VALUES ('depth', '1535', 'cardinal', '84')
INSERT INTO measurement_metadata (key, value, type, measurement_id) VALUES ('dtype', 'uint8',

Progress: 100%|██████████| 3224/3224 [00:36<00:00, 87.56it/s]


computing otsu
Transforming
computing otsu
Bounding box coordinates: zmin=0, zmax=3289, ymin=118, ymax=1670, xmin=58, xmax=248
Parameters to be inserted: 
-    file_path: \\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Juan Ignacio\probetas\5\aligned_90rotright_reslicetop.tif
-    measurementtype_id: 2
-    parent_measurement_id: 61
INSERT INTO measurements (file_path, measurementtype_id, parent_measurement_id) VALUES ('\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Juan Ignacio\probetas\5\aligned_90rotright_reslicetop.tif', '2', '61')
INSERT INTO measurement_metadata (key, value, type, measurement_id) VALUES ('height', '191', 'cardinal', '85')
INSERT INTO measurement_metadata (key, value, type, measurement_id) VALUES ('width', '3290', 'cardinal', '85')
INSERT INTO measurement_metadata (key, value, type, measurement_id) VALUES ('depth', '1553', 'cardinal', '85')
INSERT INTO measurement_metadata (key, value, type, measurement_id) VALUES ('dtype', 'uint8'

Progress: 100%|██████████| 3224/3224 [00:40<00:00, 80.33it/s]


...

computing otsu
Transforming
computing otsu
Bounding box coordinates: zmin=0, zmax=3225, ymin=20, ymax=1592, xmin=82, xmax=286
Parameters to be inserted: 
-    file_path: \\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Juan Ignacio\probetas\10\aligned_90rotright_reslicetop.tif
-    measurementtype_id: 2
-    parent_measurement_id: 31
INSERT INTO measurements (file_path, measurementtype_id, parent_measurement_id) VALUES ('\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Juan Ignacio\probetas\10\aligned_90rotright_reslicetop.tif', '2', '31')
INSERT INTO measurement_metadata (key, value, type, measurement_id) VALUES ('height', '205', 'cardinal', '88')
INSERT INTO measurement_metadata (key, value, type, measurement_id) VALUES ('width', '3226', 'cardinal', '88')
INSERT INTO measurement_metadata (key, value, type, measurement_id) VALUES ('depth', '1573', 'cardinal', '88')
INSERT INTO measurement_metadata (key, value, type, measurement_id) VALUES ('dtype', 'uint8

Progress: 100%|██████████| 3226/3226 [00:38<00:00, 82.99it/s]


computing otsu
Transforming
computing otsu
Bounding box coordinates: zmin=3, zmax=3147, ymin=10, ymax=1622, xmin=42, xmax=239
Parameters to be inserted: 
-    file_path: \\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Juan Ignacio\probetas\11\aligned_90rotright_reslicetop.tif
-    measurementtype_id: 2
-    parent_measurement_id: 32
INSERT INTO measurements (file_path, measurementtype_id, parent_measurement_id) VALUES ('\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Juan Ignacio\probetas\11\aligned_90rotright_reslicetop.tif', '2', '32')
INSERT INTO measurement_metadata (key, value, type, measurement_id) VALUES ('height', '198', 'cardinal', '89')
INSERT INTO measurement_metadata (key, value, type, measurement_id) VALUES ('width', '3145', 'cardinal', '89')
INSERT INTO measurement_metadata (key, value, type, measurement_id) VALUES ('depth', '1613', 'cardinal', '89')
INSERT INTO measurement_metadata (key, value, type, measurement_id) VALUES ('dtype', 'uint8

Progress: 100%|██████████| 3226/3226 [00:43<00:00, 74.92it/s]


computing otsu
Transforming
